# Predicción de cuotas de apuestas

En este ejercicio vamos a implementar una única red neuronal para predecir las cuotas de victoria, empate y derrota en partidos de fútbol

![img](https://www.ecestaticos.com/image/clipping/6192cd83066e196913e3045d412b937b/multan-a-la-casa-de-apuestas-digital-888-por-el-polemico-anuncio-de-carlos-sobera.jpg)

Los datos de cada casa de apuestas se encuentran en formato JSON dentro de la carpeta `data` con el nombre `bookmakerXXX.json`. Utiliza estos ficheros para crear un único dataframe con el que entrenar y probar el modelo.  
Por último, utiliza el fichero `predicciones.json` para analizar las cuotas de un único partido